In [11]:
import os
import sys
import torch
import numpy as np
import potpourri3d as pp3d
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Add the path to DiffusionNet src
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), "../../src/"))
import diffusion_net

In [12]:
# Set up parameters
model_path = "data/saved_models/sal_model.pth"  # Replace with your model path
mesh_file = "data/saliency_data/ply/bimba.ply"    # Replace with your mesh file path
input_features = 'xyz'  # or 'hks'
k_eig = 128

In [13]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [14]:
# Load the model
def load_model(model_path, input_features, device):
    C_in = {'xyz': 3, 'hks': 16}[input_features]
    model = diffusion_net.layers.DiffusionNet(
        C_in=C_in, C_out=1, C_width=128, N_block=4,
        outputs_at='vertices', dropout=True
    )
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    return model

model = load_model(model_path, input_features, device)
print("Model loaded successfully")

Model loaded successfully


In [15]:
# Prepare mesh data
def prepare_mesh_data(mesh_file, k_eig=128):
    verts, faces = pp3d.read_mesh(mesh_file)
    verts = torch.tensor(verts).float()
    faces = torch.tensor(faces)
    verts = diffusion_net.geometry.normalize_positions(verts)
    
    frames, mass, L, evals, evecs, gradX, gradY = diffusion_net.geometry.get_operators(verts, faces, k_eig=k_eig)
    return verts, faces, frames, mass, L, evals, evecs, gradX, gradY

verts, faces, frames, mass, L, evals, evecs, gradX, gradY = prepare_mesh_data(mesh_file, k_eig)
print("Mesh data prepared")

Mesh data prepared


In [16]:
# Move data to device
verts = verts.to(device)
mass = mass.to(device)
L = L.to(device)
evals = evals.to(device)
evecs = evecs.to(device)
gradX = gradX.to(device)
gradY = gradY.to(device)

In [17]:
# Make predictions
def predict(model, verts, mass, L, evals, evecs, gradX, gradY, input_features):
    with torch.no_grad():
        if input_features == 'xyz':
            features = verts
        elif input_features == 'hks':
            features = diffusion_net.geometry.compute_hks_autoscale(evals, evecs, 16)
        
        predictions = model(features, mass, L=L, evals=evals, evecs=evecs, gradX=gradX, gradY=gradY)
    return predictions.squeeze().cpu().numpy()

predictions = predict(model, verts, mass, L, evals, evecs, gradX, gradY, input_features)
print("Predictions made")

Predictions made


In [23]:
np.min(predictions), np.max(predictions)

(0.999584, 0.99999577)